In [537]:
import tabula #Extracting tables from pdf
#from pdfminer.high_level import extract_pages, ectract_text
import os
import re
from tika import parser # pip install tika

import pandas as pd
import numpy as np

In [538]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [540]:
pwd

'/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/palacet'

In [541]:
path = os.listdir("/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/palacet")

# Extract name of documents
files = [i for i in path if ".DS" not in i]

#for testing
#files = files[3:5]

In [543]:
df = pd.DataFrame(columns=['description', 'units', 'price', "date", "order_num"])

for pdf in files:
    
    order_num = pdf.replace(".pdf", "")
    
    #parser the text in the pdf (returns a dict)
    raw = parser.from_file(pdf)
    
    # Extract the content (returns a str)
    content = raw['content']
    
    # He know there is only one date in the document (and the format)
    pattern = r'\b\d{2}/\d{2}/\d{2}\b'  #meets the date format
    
    # Extract the date - returns a list of one elemnt
    date = re.findall(pattern, content)[0]
    
    #Extract tables from the pdf:
    tables = tabula.read_pdf(pdf, pages="all", pandas_options={'header': None})
    
    #We know last table is always resume of ticket.exclude:
    #tables = tables[:-1]
    
    #Information about products(main table) in different pages, are extracted as different tables:
    for table in tables:
        
        if table[0][0] == "IVA":
            continue
        
        table.rename(columns={0: "description", 1:"units", 2:"price"}, inplace=True)
        table["date"] = date
        table["order_num"] = order_num

        result = pd.concat([df, table])

        df = result
print("pdf tables extrated")

Error from tabula-java:
Error: Error: End-of-File, expected line




CalledProcessError: Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/Moni/miniconda3/envs/ironhack/lib/python3.11/site-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', 'facturas_palacet.csv']' returned non-zero exit status 1.

In [515]:
df.sample()

,description,units,pvp,date,order_num
14,Lavavajillas aloe Bosque Verde concentrado,2,"3,10 €",30/06/21,8994847


In [512]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 0 to 17
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  134 non-null    object
 1   units        129 non-null    object
 2   pvp          129 non-null    object
 3   date         134 non-null    object
 4   order_num    134 non-null    object
dtypes: object(5)
memory usage: 6.3+ KB


In [513]:
df[df["units"].isna()]

,description,units,pvp,date,order_num
0,Suavizante ropa Floral Bosque Verde concentrad...,NaN,NaN,12/12/23,16395407
0,"Toallitas limpiadoras de suelo Bosque Verde23,...",NaN,NaN,12/12/23,16395407
0,"Lejía con detergente Bosque Verde22,30 €",NaN,NaN,12/12/23,16395407
0,Bolsa de basura Bosque Verde 30L cubo mediano2...,NaN,NaN,12/12/23,16395407
0,"Friegasuelos Pino Bosque Verde líquido32,85 €",NaN,NaN,12/12/23,16395407


In [516]:
df.dropna(subset=['units'], inplace=True)

In [519]:
df.sample()

,description,units,pvp,date,order_num
15,Lavavajillas aloe Bosque Verde concentrado,1,"1,55 €",07/11/20,6947878


In [518]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129 entries, 0 to 17
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  129 non-null    object
 1   units        129 non-null    object
 2   pvp          129 non-null    object
 3   date         129 non-null    object
 4   order_num    129 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [532]:
df.sample()

,description,units,pvp,date,order_num,pvp_€,pvp/ud
13,Papel Multiusos Resistente Bosque Verde,2,"5,40 €",30/06/21,8994847,5.4,2.7


In [527]:
df["price_eu"] = df["price"].str.replace("€","").str.replace(",",".").str.strip().astype(float)
df["unit_price"] = df["price_eu"]/df["units"]

In [534]:
df["unit_price"] = df["price_eu"]/df["units"]

In [536]:
df = [["description", "units", "price_eu", "unit_price", "date", "order_num"]]

,description,units,pvp,date,order_num,pvp_€,pvp/ud,pvp/u
0,Friegasuelos pino Bosque Verde líquido,4,"2,72 €",16/09/20,6598904,2.72,0.68,0.68
1,Papel multiusos resistente Bosque Verde,1,"2,89 €",16/09/20,6598904,2.89,2.89,2.89
2,Papel higiénico doble rollo Bosque Verde,3,"11,46 €",16/09/20,6598904,11.46,3.82,3.82
3,Lavavajillas aloe Bosque Verde concentrado,4,"6,20 €",16/09/20,6598904,6.20,1.55,1.55
4,Suavizante ropa talco Bosque Verde concentrado,2,"3,58 €",16/09/20,6598904,3.58,1.79,1.79
...,...,...,...,...,...,...,...,...
13,Activador del lavado ropa blanca Bosque Verde ...,1,"1,60 €",07/11/20,6947878,1.60,1.6,1.6
14,Suavizante ropa talco Bosque Verde concentrado,2,"3,58 €",07/11/20,6947878,3.58,1.79,1.79
15,Lavavajillas aloe Bosque Verde concentrado,1,"1,55 €",07/11/20,6947878,1.55,1.55,1.55
16,Lejía con detergente Bosque Verde,1,"0,71 €",07/11/20,6947878,0.71,0.71,0.71


In [520]:
export_to_csv(df,"facturas_palacet")

DataFrame successfully exported to facturas_palacet.csv
